In [175]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [176]:
#Read in CSV
weather_df = pd.read_csv("weather_df.csv")

In [1]:
#Set coordinates for markers and define humidity for heatmap
coordinates = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity']

# Create a heatmap marker layer and display
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(coordinates, weights=humidity, max_intensity=100, point_radius=5 )
fig.add_layer(heatmap)
fig

NameError: name 'weather_df' is not defined

In [178]:
#Revise weather dataframe to cities with only ideal conditions
ideal_weather = weather_df.loc[(weather_df['Temperature'] <85) & (weather_df['Temperature'] >70)
                               & (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] <5)]
    
ideal_weather

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Date


In [170]:
#Make copy of ideal_weather df to avoid splice warning
paradise_df = ideal_weather.copy()

#Create new column in dataframe to hold hotel info
paradise_df['Hotel Name'] = ""
paradise_df["Coordinates"] = tuple(zip(paradise_df['Latitude'], paradise_df['Longitude']))
paradise_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Date,Hotel Name,Coordinates


In [171]:
# find the closest hotel to each of the city coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in paradise_df.iterrows():
    params = {
        "location": f'{row[3]},{row[4]}',
        "radius": 5000,
        "type": 'lodging',
        "key": gkey
    }
    response = requests.get(base_url, params=params).json()
    
    #Set exception for no results error, otherwise populate Hotel Name column with Hotel Name
    try:
        paradise_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except:
        paradise_df.loc[index, 'Hotel Name'] = "NaN"
        

paradise_df.head(10)

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Date,Hotel Name,Coordinates


In [172]:
paradise_df.dropna()

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Date,Hotel Name,Coordinates


In [173]:
#Create info box for marker labels
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in paradise_df.iterrows()]

#Set marker locations
marker_locations = paradise_df[['Latitude', 'Longitude']]

#Configure and display map
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))